In [1]:
import numpy as np
import os
from matplotlib import pyplot as plt
import cv2
import math
#import tensorflow as tf
import pandas as pd
import pickle
import time
from sklearn import preprocessing
import sklearn

### Load Dataset

In [2]:
epochs = 200
size = 100
#ReqLabels = [1,3,5,7,14]#,17,19,20,26,33,34,36,37,38,39]
#ReqLabels = [1,3,5,7,14,17,19,20,26,33,34,36,37,38,39]
ReqLabels = [1,3]

Train_Path = 'D:/Data/Traffic Signs/Train.csv'
Test_Path = 'D:/Data/Traffic Signs/Test.csv'
Meta_Path = 'D:/Data/Traffic Signs/Meta.csv'
Labels_Path = 'D:/Data/Traffic Signs/Labels.csv'

Labels = []
labelData = pd.read_csv(Labels_Path)
for i,j in zip(labelData['ClassId'] , labelData['SignName']) :
    if i in ReqLabels :
        Labels.append(j)
        
print(Labels)

### 30 , 50 , 70 , 100 km/h and No entry , Stop , 19,20,27,33,34,35,36,37,38,39

['Speed limit (30km/h)', 'Speed limit (60km/h)']


In [3]:
data = pd.read_csv(Meta_Path)
NoOfLabels = len(data['Path'])
print(NoOfLabels)

43


In [4]:
Count_Labels = {}
for i in ReqLabels :
    Count_Labels[i] = 0

Count_Labels

{1: 0, 3: 0}

In [5]:
def LoadData(Dir,typeLoad,count=100) :
    data = pd.read_csv(Dir)
    display(data.head())
    X , Y = [] , []
    for i,rx1,ry1,rx2,ry2,cId in zip(data['Path'],data['Roi.X1'],data['Roi.Y1'],data['Roi.X2'],data['Roi.Y2'],data['ClassId']) :
        if cId in ReqLabels :
            print(f'\r{len(X)}\t\t',end="")
            i = 'D:/Data/Traffic Signs/' + i
            img = cv2.imread(i)
            #img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            img = img[ry1:ry2,rx1:rx2]
            img = cv2.resize(img,(size,size))
            if Count_Labels[cId] <= count-1 :
                X.append(img)
                Y.append(ReqLabels.index(cId))
                if typeLoad == 'Train' or typeLoad == 'Test' :
                    Count_Labels[cId] += 1
        
    return np.array(X)/255. , np.array(Y)

In [6]:
X_train , Y_train = LoadData(Train_Path,'Train',100)

,Width,Height,Roi.X1,Roi.Y1,Roi.X2,Roi.Y2,ClassId,Path
0,27,26,5,5,22,20,20,Train/20/00020_00000_00000.png
1,28,27,5,6,23,22,20,Train/20/00020_00000_00001.png
2,29,26,6,5,24,21,20,Train/20/00020_00000_00002.png
3,28,27,5,6,23,22,20,Train/20/00020_00000_00003.png
4,28,26,5,5,23,21,20,Train/20/00020_00000_00004.png


200		

In [7]:
Count_Labels

{1: 100, 3: 100}

In [8]:
Count_Labels = {}
for i in ReqLabels :
    Count_Labels[i] = 0

Count_Labels

{1: 0, 3: 0}

In [9]:
X_test , Y_test = LoadData(Test_Path,'Test',25)

,Width,Height,Roi.X1,Roi.Y1,Roi.X2,Roi.Y2,ClassId,Path
0,53,54,6,5,48,49,16,Test/00000.png
1,42,45,5,5,36,40,1,Test/00001.png
2,48,52,6,6,43,47,38,Test/00002.png
3,27,29,5,5,22,24,33,Test/00003.png
4,60,57,5,5,55,52,11,Test/00004.png


50		

In [10]:
Count_Labels

{1: 25, 3: 25}

In [11]:
print(f'X_train shape : {X_train.shape}, Y_train shape : {Y_train.shape}')
print(f'X_test shape : {X_test.shape}, Y_test shape : {Y_test.shape}')

X_train shape : (200, 100, 100, 3), Y_train shape : (200,)
X_test shape : (50, 100, 100, 3), Y_test shape : (50,)


In [12]:
# code Train and Test data pickle dumping and loading

In [13]:
fileName = 'Model_Check.obj'
def callback(obj,fileName) :
    print('CallBack : ',fileName,'Updated')
    f = open(fileName, 'wb') 
    pickle.dump(obj, f)

* N_F = No of Neurons
* A_F = Activation Function
* __Name__ = class of the name
* __type__ = type of the class what it's do
* K_S = Kernel size
* STRIDES = how many pixels it moves at a time
* pad = no of 0's border to add
* input_shape = class input shape to train model
* output_shape = output shape of the class

---

### Activations

In [14]:
class ReLU :
    def __init__ (self) :
        self.__type__ = 'activation'
        self.__Name__ = 'ReLU'
    
    def feed(self,X) :
        self.input = X
        self.input_shape=X.shape
        self.output = np.maximum(0,X)
        self.output_shape = self.input_shape
        return self.output
    
    def feed_back(self,Z,grad_output,lr) :
        grad = Z > 0
        #print(grad.shape,grad_output.shape)
        return grad_output*grad
    
class Softmax :
    def __init__ (self) :
        self.__type__ = 'activation'
        self.__Name__ = 'Softmax'
    
    def feed(self,X) :
        self.input = X
        e_x = np.exp(X-np.max(X))
        self.output = e_x/e_x.sum()
        return np.exp(X)/np.sum(np.exp(X))
    
    def grad_feed(self,X) :
        e_x = np.exp(X)
        return (e_x/e_x.sum()) - (np.power(e_x,2)/np.power(e_x.sum(),2))
    
    def feed_back(self,Z,grad_output,lr) :
        e_x = np.exp(Z)
        out = e_x/ex.sum()
        grad = (e_x/e_x.sum()) - (np.power(e_x,2)/np.power(e_x.sum(),2))
        #grad = e_x/e_x.sum()**2 - (e_x**2/(e_x.sum()**2))
        return grad_output*grad
    
class Sigmoid :
    def __init__ (self) :
        self.__type__ = 'activation'
        self.__Name__ = 'Sigmoid'
    
    def feed(self,X) :
        self.input = X
        self.output = 1/(1+np.exp(-X))
        return self.output
    
    def grad_feed(self,X) :
        out = 1/(1+np.exp(-X))
        return out*(1-out)
    
    def feed_back(self,Z,grad_output,lr) :
        out = 1/(1+np.exp(-Z))
        grad = out*((1-out)**2)
        return grad_output*grad

## Layers

`Convolution` Layer

In [15]:
class Conv2D :
    
    def __init__ (self,N_F,K_S,A_F,input_shape,STRIDES=1,pad=0) :
        self.__Name__ = 'Conv2D'
        self.__type__ = 'conv'
        self.N_F = N_F
        self.K_S = K_S
        self.STRIDES = STRIDES
        self.A_F = A_F
        self.pad = pad
        if (len(input_shape) == 2) :
            self.input_shape = input_shape+(1,)
        else :
            self.input_shape = input_shape
        self.Filters = np.random.normal(size=(self.N_F,self.input_shape[-1],self.K_S,self.K_S))
        self.W = int((self.input_shape[0]-self.K_S+2*self.pad)/self.STRIDES) + 1
        self.H = int((self.input_shape[1]-self.K_S+2*self.pad)/self.STRIDES) + 1
        self.D = self.N_F
        self.bias = np.random.randint(-1,2,(self.N_F))
        self.output_shape = (self.W,self.H,self.D)

    def feed(self,X) :
        output = np.zeros((self.W,self.H,self.D))
        for i in range(0,self.W,self.STRIDES) :
            for j in range(0,self.H,self.STRIDES) :
                cur_reg = X[i:i+self.K_S,j:j+self.K_S].T*self.Filters
                output[i,j] = np.sum(cur_reg,axis=(1,2,3)) + self.bias

        return output
    
    def feed_back(self,X) :
        return None

    def plotImg(self,X=None) :
        if X is None :
            X = self.output
        Filter_SIZE = int(X.shape[-1]**(1/2))
        _, axs = plt.subplots(Filter_SIZE,Filter_SIZE, figsize=(8,8))
        axs = axs.flatten()
        for i , ax in enumerate(axs) :
            img = X[:,:,i]
            ax.axis('off')
            ax.imshow(img)
        plt.show()
        
    def Summary(self) :
        l = len(self.__Name__)
        print(f'{self.__Name__}',' '*(20-l),self.input_shape,' '*(20-len(str(self.input_shape))),self.output_shape)
    
# conv = Conv2D(16,3,'ReLU',input_shape=X_train[0].shape)
# conv.Summary()
# output = conv.feed(X_train[0])
# conv.plotImg(output)

`MaxPooling` Layer

In [16]:
class MaxPool2D :
    
    def __init__ (self,K_S,input_shape,STRIDES=2,pad=0) :
        self.__Name__ = 'MaxPool2D'
        self.__type__ = 'pool'
        self.K_S = K_S
        self.STRIDES = STRIDES
        self.A_F = None
        self.input_shape = input_shape
        self.output_shape = (int((self.input_shape[0]-self.K_S+1)/self.STRIDES),int((self.input_shape[0]-self.K_S+1)/self.STRIDES),self.input_shape[-1])
        
    def feed(self,X) :
        self.input = X
        W,H,D = self.output_shape
        self.output = np.zeros((W,H,D))
        w , h = 0 , 0
        for i in range(0,self.input_shape[0]-self.K_S,self.STRIDES) :
            for j in range(0,self.input_shape[1]-self.K_S,self.STRIDES) :
                #print(X[i:i+self.K_S,j:j+self.K_S])
                self.output[w,h] = np.max(np.max(X[i:i+self.K_S,j:j+self.K_S].T,axis=1),axis=1)
                # (98,10) , (100,12)
                h += 1
            h = 0
            w += 1
            
        return self.output
    
    def plotImg(self,X=None) :
        if X is None :
            X = self.output
        Filter_SIZE = int(X.shape[-1]**(1/2))
        _, axs = plt.subplots(Filter_SIZE,Filter_SIZE, figsize=(8,8))
        axs = axs.flatten()
        for i , ax in enumerate(axs) :
            img = X[:,:,i]
            ax.axis('off')
            ax.imshow(img)
        plt.show()
        
    def Summary(self) :
        l = len(self.__Name__)
        print(f'{self.__Name__}',' '*(20-l),self.input_shape,' '*(20-len(str(self.input_shape))),self.output_shape)
        
# maxpool = MaxPool2D(3,input_shape=conv.output_shape)
# maxpool.Summary()
# output = maxpool.feed(conv.output)
# maxpool.plotImg(output)

`Average Pooling` Layer

In [17]:
class AvgPool2D :
    
    def __init__ (self,K_S,input_shape,STRIDES=2) :
        self.__Name__ = 'AvgPool2D'
        self.__type__ = 'pool'
        self.K_S = K_S
        self.STRIDES = STRIDES
        self.A_F = None
        self.input_shape = input_shape
        self.output_shape = (int((self.input_shape[0]-self.K_S+1)/self.STRIDES),int((self.input_shape[0]-self.K_S+1)/self.STRIDES),self.input_shape[-1])
        
    def feed(self,X) :
        self.input = X
        W,H,D = self.output_shape
        self.output = np.zeros((W,H,D))
        w , h = 0 , 0
        for i in range(0,self.input_shape[0]-self.K_S,self.STRIDES) :
            for j in range(0,self.input_shape[1]-self.K_S,self.STRIDES) :
                self.output[w,h] = np.average(np.average(X[i:i+self.K_S,j:j+self.K_S].T,axis=1),axis=1)
                h += 1
            h = 0
            w += 1
            
        return self.output
    
    def plotImg(self,X=None) :
        if X is None :
            X = self.output
        Filter_SIZE = int(X.shape[-1]**(1/2))
        _, axs = plt.subplots(Filter_SIZE,Filter_SIZE, figsize=(8,8))
        axs = axs.flatten()
        for i , ax in enumerate(axs) :
            img = X[:,:,i]
            #ax.axis('off')
            ax.imshow(img)
        plt.show()
        
    def Summary(self) :
        l = len(self.__Name__)
        print(f'{self.__Name__}',' '*(20-l),self.input_shape,' '*(20-len(str(self.input_shape))),self.output_shape)
        
        
# avgpool = AvgPool2D(3,input_shape=conv.output_shape)
# avgpool.Summary()
# output = avgpool.feed(conv.output)
# maxpool.plotImg(output)

### `Flatten` Layer

In [24]:
class Flatten :
    
    """
        Flatten class is used to convert data into single dimension
    """
    
    def __init__ (self,input_shape=None,output_shape=None,__Name__='Flatten') : ### Constructor called when we create object 
        self.__Name__ = __Name__ ### Defining __Name__ variable with Flatten
        self.__type__ = 'flat' ### Defining __type__ variable
        self.input_shape = input_shape
        self.A_F = None
        re = 1
        if output_shape is None :
            for i in input_shape :
                re *= i
            self.output_shape = re
        else :
            self.output_shape = output_shape
        
    def feed(self,X) : ### feed function is used to transforms data into single dimension
        #self.input = X
        self.output = X.ravel() ### ravel is used to convert data into single dimensoin or flattens data
        return self.output
    
    def feed_back(self,X) :
        return X.reshape(self.input_shape)
    
    def Summary(self) :
        l = len(self.__Name__)
        print(f'{self.__Name__}',' '*(20-l),self.input_shape,' '*(20-len(str(self.input_shape))),self.output_shape)
        
# flat = Flatten(input_shape = (2,3))
# out = np.random.randint(0,2,(2,3))
# print(out)
# out = flat.feed(out)
# print(out)
# out = flat.feed_back(out)
# print(out)

[[1 1 1]
 [1 0 0]]
[1 1 1 1 0 0]
[[1 1 1]
 [1 0 0]]


### `Dense` Layer

In [19]:
class Dense :
    
    def __init__ (self,input_shape,N_F,A_F=None,wt=None,bias=None,output_shape=None,__Name__='Dense') :
        
        """
            Wt and Bias range [-2.4/No of nodes , 2.4/No of nodes] Proposed by range
        """
        self.__Name__ = __Name__
        self.__type__ = 'dense'
        self.input_shape = input_shape
        self.N_F = N_F
        self.A_F = A_F
        if output_shape is None :
            self.output_shape = N_F
        else :
            self.output_shape = output_shape
        if wt is None :
            #self.weights = np.random.uniform(self.input_shape, self.output_shape) / np.sqrt(self.input_shape + self.output_shape)
            self.weights = np.random.uniform(-2.4/self.N_F,2.4/self.N_F,(self.input_shape, self.output_shape))
        else :
            self.weights = wt
        if bias is None :
            #self.bias = np.random.randn(1, self.output_shape) / np.sqrt(self.input_shape + self.output_shape)
            self.bias = np.random.uniform(-2.4/ self.N_F,2.4/ self.N_F,(1, self.output_shape)) / self.N_F
        else :
            self.bias = bias
        
    def feed(self,X) :
        if X.shape[0] != 1 :
            output = []
            output.append(X)
            self.input = np.array(output)
        else :
            self.input = X
        self.output = np.matmul(X,self.weights) + self.bias
        self.output = sklearn.preprocessing.normalize(self.output)
        return self.output
    
    def feed_back(self, Z , output_error, learning_rate=1e-03,deacy=1e-02):
        input_error = np.dot(output_error, self.weights.T)
        #output_error = output_error.mean(axis=0)*Z.T.shape[0]
        weights_error = np.dot(Z.T, output_error)
        self.weights = self.weights - learning_rate * weights_error
        self.bias = self.bias - learning_rate * output_error
#         self.weights = (1-decay)*self.weights - learning_rate * weights_error
#         self.bias = (1-decay)*self.bias - learning_rate * output_error
        return input_error
    
    def Summary(self) :
        l = len(self.__Name__)
        print(f'{self.__Name__}',' '*(20-l),self.input_shape,' '*(20-len(str(self.input_shape))),self.output_shape)

In [20]:
class Sequential :
    
    """
        Sequential is a class which is used to stack layers of model and to fit , predict , predicting classes of our given i/p
    """
    
    def __init__ (self) :
        self.Layers = []
        self.input_shape = None
        self.Activations = []
        self.acc = []
        self.val_acc = []
        self.error = []
        self.val_error = []
        self.id = 1
        
    def add(self,Layer) :
        boo = False
        for layer in self.Layers :
            if Layer.__type__ == layer.__type__ :
                boo = True
                if '_' not in Layer.__Name__ :
                    Layer.__Name__ += '_'+str(self.id)
                name,k = Layer.__Name__.split('_')
                Layer.__Name__ = name+'_'+str(int(k)+1)
        if not boo :
            if '_' not in Layer.__Name__ :
                Layer.__Name__ += '_'+str(self.id)
        
        self.Layers.append(Layer)
        if Layer.__type__ != 'activation' :
            if self.input_shape is None :
                self.input_shape = Layer.input_shape
            self.output_shape = Layer.output_shape
        if Layer.A_F is not None :
            if Layer.A_F.lower() == 'softmax' :
                self.Activations.append(Softmax())
            elif Layer.A_F.lower() == 'sigmoid' :
                self.Activations.append(Sigmoid())
            else :
                self.Activations.append(ReLU())
        else :
            self.Activations.append(None)
        
    def compile(self,loss='cross_entropy',metrics=['acc']) :
        self.loss = loss
        self.metrics = metrics
        
    def shuffle(self,input_,target) :
        n = np.random.randint((len(input_),1))
        c = 0
        for i in n :
            input_[c] , input_[i] = input_[i] , input_[c]
            target[c] , target[i] = target[i] , target[c]
            c += 1
        return input_ , target
        
    def one_hot_encode(self,l) :
        Labels = np.zeros(self.Layers[-1].output_shape)
        Labels[l] = 1
#         for i,label in enumerate(labels) :
#             Labels[i][label] = 1
        return Labels
        
    def fit(self,train_data,valid_data=None,validation_split=.1,epochs=10,lr=1e-02,decay=1e-03,batch_size=1) :
        self.epochs = epochs
        input_ , Total , target , No_of_outs = None , None , None , None
        val_input , val_target = None , None
        if train_data is None :
            raise ValueError('Training Data Required')
        else :
            input_ = train_data[0]
            Total = train_data[1]
            target = Total
            No_of_outs = len(set(target))
        N = len(input_)
        
        if valid_data is None :
            if validation_split != 0 :
                n = int(len(input_)*(1-validation_split))
                K = np.random.randint(0,N,(N))
                for i in range(N-1) :
                    train_data[0][K[i]] = train_data[0][K[i+1]]
                    train_data[1][K[i]] = train_data[1][K[i+1]]
                input_ , target = train_data[0][:n] , Total[:n]
                val_input , val_target = train_data[0][n:] , Total[n:]
        else :
            val_input , val_target = valid_data[0] , valid_data[1]
        
        print('\nModel Fitting\n')
        
        N = len(input_)
        
#         input_ = preprocessing.normalize(input_,axis=0)
#         val_input = preprocessing.normalize(val_input,axis=0)
        
        for ep in range(epochs) :
            start_ep = time.time()
            error = 0
            acc = 0
            input_ , target = self.shuffle(input_,target)
            
            print(f'\nepoch : {ep+1}/{epochs}')
            
            for c,(X,Y) in enumerate(zip(input_,target)) :
                
                #print(f'\r[','='*int(c*50/N),'>','.'*(50-int(c*50/N)),']'+f' Error : %.4f' %(error),end="")
                #print(c)
                
                
                L_INPUTS , L_OUTPUTS ,  A_INPUTS , A_OUTPUTS = [] , [] , [] , []
                
                output = X
                loss = None
                out_err = None
                
                """
                    Forward Feeding [ Z = W*X + B ]
                """
                for layer , activation in zip(self.Layers,self.Activations) :
                    L_INPUTS.append(output)
                    output = layer.feed(output) # Feeding to Layer
                    L_OUTPUTS.append(output)
                    if activation is not None :
                        A_INPUTS.append(output)
                        output = activation.feed(output) # applying activation to output of the Layers
                        A_OUTPUTS.append(output)
                        
                activation_output = self.Activations[-1].output # a-l
                output = self.Layers[-1].output # z-l

                """
                    Loss Calculation or Output Error
                """
                
                if self.loss == 'cross_entropy' :
                    #loss = self.crossentropy(activation_output,Y)
                    loss = self.cat_crossentropy(activation_output,Y)
                    if math.isnan(loss) :
                        model.epochs = ep-1
                        return None
                    grad_activation = self.Activations[-1].grad_feed(output)
                    out_err = self.grad_crossentropy(output,Y)*grad_activation
                
                """
                    Backward Feeding
                """
                
                try :
                    for i in range(1,len(self.Layers)-1) :
                        if self.Layers[-i].__Name__[0] == 'D' :
                            #print(self.Layers[-i].__Name__)
                            if self.Activations[-i].__Name__ != 'Softmax' :
                                out_err = self.Activations[-i].feed_back(A_INPUTS[-i],out_err,lr)
                                #print(self.Activations[-i].__Name__)
                            out_err = self.Layers[-i].feed_back(L_INPUTS[-i],out_err,lr,decay)
                        else :
                            break
                except :
                    pass
                
                error = np.mean(loss)
                print('\rerror=%f' % (error),end="")
                #print('\rerror=%f -> c=%f' % (error,c),end="")
                
            """
                Accuracy measuring at every epoch
            """
            accuracy = sum([y == np.argmax(model.predict(x)) for x, y in zip(input_, target)]) / len(input_)
            self.acc.append(accuracy)
            self.error.append(error)
            
            if 'acc' in self.metrics :
                val_accuracy = sum([y == np.argmax(model.predict(x)) for x, y in zip(val_input, val_target)]) / len(val_input)
                self.val_acc.append(val_accuracy)
                print(' acc=%f , val_acc=%f' % (accuracy , val_accuracy))
            else :
                print('\racc=%f' % (accuracy))
                
            callback(model,fileName)
            
            end_ep = time.time()
            
            print(f'Time Taken for epoch {ep+1} : {end_ep-start_ep}s')
            
        return None
            
    def mse(self,y_true, y_pred):
        return np.mean(np.power(y_true - y_pred, 2))
    
    def mse_prime(self,y_true, y_pred):
        return 2 * (y_pred - y_true) / y_pred.size
    
    def transfer_derivative(self,output):
        return output * (1.0 - output)
    
    def binary_crossentropy(self,pred,Truth) :
        GroundTruth = np.zeros(self.No_of_outs)
        GroundTruth[Truth] = 1
        return -np.mean(GroundTruth*np.log(pred)+(1-GroundTruth)*np.log(1-pred))
    
    def binary_grad_crossentropy(self,pred,Truth) :
        GroundTruth = np.zeros(self.No_of_outs)
        GroundTruth[Truth] = 1
        return -((GroundTruth/pred)-((1-GroundTruth)/(1-pred)))
    
    def cat_crossentropy(self,pred,Truth) :
        Truth = self.one_hot_encode(Truth)
#         a = Truth/pred
#         b = self.Activations[-1].grad_feed(self.Layers[-1].output)[0]
#         print(Truth,pred,a,b)
#         print(np.dot(a,b.T))
        return np.dot((Truth/pred),self.Activations[-1].grad_feed(self.Layers[-1].output)[0])

    def grad_cat_crossentropy(self,pred,Truth) :
        Truth = self.one_hot_encode(Truth)
        a = Truth/pred
        b = self.Activations[-1].grad_feed(self.Layers[-1].output)[0]
        print(Truth,pred,a,b)
        print(np.dot(a,b.T))
        return np.dot((Truth/pred),self.Activations[-1].grad_feed(self.Layers[-1].output)[0])
    
    def crossentropy(self,logits,reference_answers):
        return - logits[0][reference_answers] + np.log(np.sum(np.exp(logits),axis=-1))
    
    def grad_crossentropy(self,logits,reference_answers):
        ones_for_answers = np.zeros_like(logits)
        ones_for_answers[np.arange(len(logits)),reference_answers] = 1
        softmax = np.exp(logits) / np.exp(logits).sum(axis=-1,keepdims=True)
        return (- ones_for_answers + softmax) / logits.shape[0]
    
    def showImg(self,X) :
        plt.imshow(X)
        plt.show()
    
    def predict(self,X):
        outputs = []
        if X.shape == model.input_shape :
            output = X
            for layer , activation in zip(self.Layers,self.Activations) :
                output = layer.feed(output)
                if activation is not None :
                    output = activation.feed(output)
            outputs.append(output)
        else :
            for output in X :
                for layer , activation in zip(self.Layers,self.Activations) :
                    output = layer.feed(output)
                    if activation is not None :
                        output = activation.feed(output)
                outputs.append(output)
        return np.array(outputs)
    
    def pred_class(self,X) :
        classes = []
        if X.shape == model.input_shape :
            output = self.predict(X)
            return np.argmax(output)
        else :
            for output in X :
                output = self.predict(output)
                classes.append(np.argmax(output))
            return np.array(classes)
    
    def Summary(self) :
        print('='*60)
        print('Model Summary')
        print('_'*60)
        print('Layers',' '*(20-len('Layers')),'Input Shape',' '*(20-len('Input Shape')),'Output Shape',' '*(20-len('Output Shape')))
        print('='*60)
        for Layer in self.Layers :
            if Layer.__type__ != 'activation' :
                Layer.Summary()
                print('_'*60)
        print('='*60)

In [21]:
model = Sequential()
model.add(Conv2D(3,5,input_shape=X_train[0].shape,A_F='ReLU'))
model.add(MaxPool2D(5,input_shape=model.output_shape))
model.add(Flatten(input_shape=model.output_shape))
#model.add(Flatten(input_shape=X_train[0].shape))
model.add(Dense(input_shape=model.output_shape,N_F=100,A_F='ReLU')) ### Input Layer
#model.add(Dense(input_shape=model.output_shape,N_F=32,A_F='ReLU')) ## Hidden Layer
model.add(Dense(input_shape=model.output_shape,N_F=len(set(Y_train)),A_F='Sigmoid')) ### Output Layer
model.compile(loss='cross_entropy',metrics=['acc'])

model.Summary()

Model Summary
____________________________________________________________
Layers                Input Shape           Output Shape         
Conv2D_1              (100, 100, 3)         (96, 96, 3)
____________________________________________________________
MaxPool2D_1           (96, 96, 3)           (46, 46, 3)
____________________________________________________________
Flatten_1             (46, 46, 3)           6348
____________________________________________________________
Dense_1               6348                  100
____________________________________________________________
Dense_2               100                   2
____________________________________________________________


In [22]:
model.fit(train_data=(X_train,Y_train),epochs=epochs,lr=0.01,valid_data=(X_test,Y_test),batch_size=1) #,validation_split=.1


Model Fitting


epoch : 1/200
error=0.666601 acc=0.340000 , val_acc=0.320000
CallBack :  Model_Check.obj Updated
Time Taken for epoch 1 : 171.40432476997375s

epoch : 2/200
error=0.669143 acc=0.345000 , val_acc=0.300000
CallBack :  Model_Check.obj Updated
Time Taken for epoch 2 : 171.00619101524353s

epoch : 3/200
error=0.671532 acc=0.340000 , val_acc=0.300000
CallBack :  Model_Check.obj Updated
Time Taken for epoch 3 : 177.44751358032227s

epoch : 4/200
error=0.673759 acc=0.340000 , val_acc=0.300000
CallBack :  Model_Check.obj Updated
Time Taken for epoch 4 : 172.0234875679016s

epoch : 5/200
error=0.675842 acc=0.340000 , val_acc=0.300000
CallBack :  Model_Check.obj Updated
Time Taken for epoch 5 : 168.9220471382141s

epoch : 6/200
error=0.677770 acc=0.350000 , val_acc=0.280000
CallBack :  Model_Check.obj Updated
Time Taken for epoch 6 : 169.39387130737305s

epoch : 7/200
error=0.679575 acc=0.345000 , val_acc=0.280000
CallBack :  Model_Check.obj Updated
Time Taken for epoch 7 : 168.9

KeyboardInterrupt: 

In [ ]:
### Call Back to save model at every epoch

In [ ]:
print(model.acc)

In [ ]:
def SaveModel(obj,fileName) :
    f = open(fileName, 'wb') 
    pickle.dump(obj, f)
    
def loadModel(fileName) :
    f = open(fileName, 'rb') 
    model = pickle.load(f)
    return model

In [ ]:
loadModel(fileName)

In [ ]:
plt.plot(range(model.epochs),model.acc,c='r',label='acc')
plt.plot(range(model.epochs),model.val_acc,c='b',label='val_acc')
plt.legend()
plt.show()

In [ ]:
plt.plot(range(model.epochs),model.error,c='black',label='error')
plt.legend()
plt.show()

In [ ]:
c = 0
for i in range(len(X_test)) :
    print(model.pred_class(X_test[i]),Y_test[i])
    if model.pred_class(X_test[i]) == Y_test[i] :
        c += 1
print(c*100/len(X_test))

In [ ]:
model.pred_class(X_test)

In [ ]:
model.pred_class(X_train[0])

In [ ]:
c = 0
for i in range(len(X_train)) :
    #print(model.pred_class(X_train[i]),Y_train[i])
    if model.pred_class(X_train[i]) == Y_train[i] :
        c += 1
print(c*100/len(X_train))

In [ ]:
img = X_train[1]
print(model.predict(img))
print(Labels[model.pred_class(img)])
plt.imshow(img)
plt.show()

In [ ]:
img = X_train[10]
print(model.predict(img))
print(Labels[model.pred_class(img)])
plt.imshow(img)
plt.show()

In [ ]:
Dir = 'D:/Data/Traffic Signs/Meta'
for i in os.listdir(Dir) :
    print(i)
    img = cv2.imread(Dir+'/'+i)
    img = cv2.resize(img,(size,size))/255.
    test_data = []
    test_data.append(img)
    test_data = np.array(test_data)
    
    plt.imshow(img)
    plt.show()
    
    print(Labels[model.pred_class(test_data)[0]])

In [ ]:
Labels

In [ ]:
w = np.array([[1,2,3]])
x = np.array([1,2,3])

In [ ]:
a = (w*x)

In [ ]:
a

a = np.array([1,2,3])

In [ ]:
e_x = np.exp(a)

In [ ]:
e_x

In [ ]:
e__ = e_x / np.sum(e_x)

In [ ]:
e__

In [ ]:
emp = []
for i in range(len(e_x)) :
    res = (e_x[i]*np.sum(e_x) - e_x[i]**2)/np.sum(e_x)**2
    emp.append(res)

In [ ]:
emp

In [ ]:
a

y = wx+b
E = -sum(T.log(y))
dE/dW = -[T0*[1-(e**Y/sum(e**Y))]*X]